# EEG Preprocessing with `ica_xtra.py`  
### *A single-subject tutorial for 280-channel EGI data*

This notebook shows you how to clean one EEG recording using a robust, automated pipeline that:
- Detects and interpolates bad channels (twice!)
- Removes artifacts with ICA (blinks, heartbeats, muscle)
- Preserves all **281 channels** (`E1–E280` + `Cz`)
- Works on **FIF or MFF** files
- Uses a **shared montage** (`.gpsc`) for all subjects




In [ ]:
## 📦 Requirements

# Install these packages first:

# pip install mne scipy numpy matplotlib
# pip install  python-picard
# pip install onnxruntime                   # ← essential to make icalabel work
# pip install mne-icalabel


# Optional to make icalabel work: !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


## 🧪 Step 1: Import the Library

In [ ]:
# Import your preprocessing pipeline
from ica_xtra import run_preprocessing_pipeline
from pathlib import Path


# Define paths 
INPUT_FIF = "rest_off_sub-02_c_eeg.fif"
GPS_FILE  = "ghw280_from_egig.gpsc"
OUTPUT_DIR = "sub-02"  # Save output in current folder

subject_id = "sub-02"

## ▶️ Step 2: Run the Pipeline


In [ ]:
# Run preprocessing!
run_preprocessing_pipeline(
    subject=subject_id,
    input_path=INPUT_FIF,
    gpsc_file=GPS_FILE,           
    base_output_path=OUTPUT_DIR,
    input_format="fif",
    
    # Filtering
    apply_highpass=True,
    apply_lowpass=True,
    apply_notch=True, 
    line_freq=60.0,
    
    # Bad channel detection (Z_score)
    pre_ica_mad_threshold=3.5,
    post_ica_mad_threshold=5.0,
    
    # Output control
    append_subject_to_output=False,
    plot=True,
    log_to_file=True
)

print("✅ Done! Check your folder for:")
print(f"  - {subject_id}_eeg_ica_cleaned_raw.fif")
print(f"  - {subject_id}_preproc_log.txt")
print(f"  - plots/ (diagnostic images)")

## 🔍 Step 3: Inspect Results

In [7]:
import mne

# Load cleaned data
raw = mne.io.read_raw_fif("sub-02\sub-02_eeg_ica_cleaned_raw.fif", preload=True)

# Verify
raw.info

<Info | 13 non-empty values
 bads: []
 ch_names: E1, E2, E3, E4, E5, E6, E7, E8, E9, E10, E11, E12, E13, E14, ...
 chs: 281 EEG, 8 Stimulus
 custom_ref_applied: True
 device_info: 1 item (dict)
 dig: 284 items (3 Cardinal, 281 EEG)
 file_id: 4 items (dict)
 highpass: 1.0 Hz
 lowpass: 100.0 Hz
 meas_date: 2025-01-08 20:06:35 UTC
 meas_id: 4 items (dict)
 nchan: 289
 projs: []
 sfreq: 500.0 Hz
 utc_offset: +0000
>

In [ ]:
# Plot 10 seconds

raw.plot(n_channels=40);

## How It Works (Simplified)

1. **Load & Rename**: `1` → `E1`, `REF CZ` → `Cz`
2. **Apply Montage**: From your shared `.gpsc` file
3. **Filter**: Notch (60 Hz), optional high/low-pass
4. **Detect Bad Channels** (Stage 1):  
   - Uses MAD (median absolute deviation)  
   - Protects key channels (e.g., `E31`, `E19` for EOG)

5. **Run ICA**:  
   - Creates virtual EOG/ECG (`vVEOG = E31 - E19`)  
   - Removes blink, heartbeat, muscle artifacts
6. **Detect Bad Channels** (Stage 2):  
   - More sensitive, no protection → catches residual noise
7. **Interpolate All Bad Channels** → final output has **281 clean channels**
8. **Save**: Cleaned FIF + logs + plots

